# Saisie de données incohérentes



Points traités dans ce notebook :

- Mise en place de notre environnement
- Effectuer un prétraitement préliminaire du texte
- Utiliser l'appariement flou pour corriger les données incohérentes.

# Mise en place de notre environnement

La première chose à faire est de charger les bibliothèques que nous utiliserons. Mais pas nos jeux de données : nous y reviendrons plus tard !

In [12]:
# librairie
import pandas as pd
import numpy as np

# librairie utile
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# reproductibilité
np.random.seed(0)

Lorsque j'ai essayé de lire le fichier PakistanSuicideAttacks Ver 11 (30-November-2017).csvfile la première fois, j'ai obtenu une erreur d'encodage de caractères, je vais donc vérifier rapidement quel devrait être l'encodage...

In [13]:
# regarder les dix mille premiers octets pour deviner le codage des caractères
with open("/Users/jl/Downloads/PakistanSuicideAttacks Ver 11 (30-November-2017).csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# vérifier le codage des caractères
print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


Le lire avec l'encodage correct.

In [14]:
# lire dans notre date
suicide_attacks = pd.read_csv("/Users/jl/Downloads/PakistanSuicideAttacks Ver 11 (30-November-2017).csv", 
                              encoding='Windows-1252')

à présent, on peut commencer



# Effectuer un prétraitement préliminaire du texte


Pour cet exercice, je souhaite nettoyer la colonne "Ville" pour m'assurer qu'il n'y a pas d'incohérences dans la saisie des données. Nous pourrions bien sûr vérifier chaque ligne à la main et corriger les incohérences lorsque nous les trouvons. Mais il existe un moyen plus efficace d'y parvenir !

In [15]:
# obtenir toutes les valeurs uniques dans la colonne 'Ville'
cities = suicide_attacks['City'].unique()

# les trier par ordre alphabétique et regarder de plus près
cities.sort()
cities

array(['ATTOCK', 'Attock ', 'Bajaur Agency', 'Bannu', 'Bhakkar ', 'Buner',
       'Chakwal ', 'Chaman', 'Charsadda', 'Charsadda ', 'D. I Khan',
       'D.G Khan', 'D.G Khan ', 'D.I Khan', 'D.I Khan ', 'Dara Adam Khel',
       'Dara Adam khel', 'Fateh Jang', 'Ghallanai, Mohmand Agency ',
       'Gujrat', 'Hangu', 'Haripur', 'Hayatabad', 'Islamabad',
       'Islamabad ', 'Jacobabad', 'KURRAM AGENCY', 'Karachi', 'Karachi ',
       'Karak', 'Khanewal', 'Khuzdar', 'Khyber Agency', 'Khyber Agency ',
       'Kohat', 'Kohat ', 'Kuram Agency ', 'Lahore', 'Lahore ',
       'Lakki Marwat', 'Lakki marwat', 'Lasbela', 'Lower Dir', 'MULTAN',
       'Malakand ', 'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mohmand Agency ', 'Mohmand agency', 'Mosal Kor, Mohmand Agency',
       'Multan', 'Muzaffarabad', 'North Waziristan', 'North waziristan',
       'Nowshehra', 'Orakzai Agency', 'Peshawar', 'Peshawar ', 'Pishin',
       'Poonch', 'Quetta', 'Quetta ', 'Rawalpindi', 'Sargodha',
       'Sehwan town',

Rien qu'en regardant cela, je peux voir quelques problèmes dus à une saisie incohérente des données : "Lahore" et "Lahore", par exemple, ou "Lakki Marwat" et "Lakki marwat".

La première chose que je vais faire est de mettre tout en minuscule (je pourrai le faire à la fin si je le souhaite) et de supprimer les espaces blancs au début et à la fin des cellules.

Les incohérences dans les majuscules et les espaces blancs en fin de ligne sont très courantes dans les données textuelles et vous pouvez corriger un bon 80 % de vos incohérences de saisie de données textuelles en procédant ainsi.

In [16]:
# convertir en minuscules
suicide_attacks['City'] = suicide_attacks['City'].str.lower()

# supprimer les espaces blancs à la fin du texte
suicide_attacks['City'] = suicide_attacks['City'].str.strip()

Nous allons maintenant nous attaquer à des incohérences plus difficiles.

# Utiliser la correspondance floue pour corriger les incohérences dans la saisie des données

regardons à présent la colonne ville et voyons s'il y a d'autres nettoyages de données à faire.

In [17]:
# obtenir toutes les valeurs uniques dans la colonne 'Ville'
cities = suicide_attacks['City'].unique()

# les trier par ordre alphabétique et les examiner de plus près
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd. i khan', 'd.g khan', 'd.i khan',
       'dara adam khel', 'fateh jang', 'ghallanai, mohmand agency',
       'gujrat', 'hangu', 'haripur', 'hayatabad', 'islamabad',
       'jacobabad', 'karachi', 'karak', 'khanewal', 'khuzdar',
       'khyber agency', 'kohat', 'kuram agency', 'kurram agency',
       'lahore', 'lakki marwat', 'lasbela', 'lower dir', 'malakand',
       'mansehra', 'mardan', 'mohmand agency',
       'mosal kor, mohmand agency', 'multan', 'muzaffarabad',
       'north waziristan', 'nowshehra', 'orakzai agency', 'peshawar',
       'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=

Il semble qu'il reste quelques incohérences : "d. i khan" et "d.i khan" devraient être identiques. (J'ai vérifié et 'd.g khan' est une ville distincte, je ne devrais donc pas les combiner).

Je vais utiliser le paquet fuzzywuzzy pour m'aider à identifier les chaînes les plus proches les unes des autres. Ce dataset est suffisamment petit pour que nous puissions probablement corriger les erreurs à la main, mais cette approche n'est pas adaptée. Automatiser les choses le plus tôt possible est généralement une bonne idée.

Fuzzy matching (=Correspondance floue) : processus consistant à trouver automatiquement des chaînes de texte très similaires à la chaîne cible.
En général, on considère qu'une chaîne est d'autant plus proche d'une autre qu'il y a moins de caractères à modifier pour transformer une chaîne en une autre.
Ainsi, "apple" et "snapple" sont à deux changements l'un de l'autre (ajouter "s" et "n"), tandis que "in" et "on" sont à un changement l'un de l'autre (remplacer "i" par "o").
Vous ne pourrez pas toujours compter à 100 % sur la correspondance floue, mais elle nous permettra généralement de gagner au moins un peu de temps.

Fuzzywuzzy renvoie un ratio à partir de deux chaînes de caractères. Plus le ratio est proche de 100, plus la distance d'édition entre les deux chaînes est faible. Ici, nous allons obtenir les dix chaînes de notre liste de villes qui ont la distance la plus proche de "d.i khan".

In [18]:
# obtenir les 10 correspondances les plus proches de "d.i khan"
matches = fuzzywuzzy.process.extract("d.i khan", cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# regardons le résultat
matches

[('d. i khan', 100),
 ('d.i khan', 100),
 ('d.g khan', 88),
 ('khanewal', 50),
 ('sudhanoti', 47),
 ('hangu', 46),
 ('kohat', 46),
 ('dara adam khel', 45),
 ('chaman', 43),
 ('mardan', 43)]

Nous pouvons constater que deux des éléments des villes sont très proches de "d.i khan" : "d. i khan" et "d.i khan". Nous pouvons également voir que "d.g khan", qui est une ville distincte, a un ratio de 88. Puisque nous ne voulons pas remplacer "d.g khan" par "d.i khan", remplaçons toutes les lignes de notre colonne Ville qui ont un ratio > 90 par "d. i khan".

Pour ce faire, je vais écrire une fonction.

Il est conseillé d'écrire une fonction d'usage général et dont je pourrais réutiliser si je pense devoir effectuer une tâche spécifique plusieurs fois

Cela permet d'éviter de copier et coller du code trop souvent, ce qui nous fait gagner du temps et peut nous aider à éviter les erreurs.

In [19]:
# fonction permettant de remplacer les lignes de la colonne fournie de la base de données fournie
# qui correspondent à la chaîne fournie au-dessus du ratio fourni par la chaîne fournie
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # obtient une liste de chaînes uniques
    strings = df[column].unique()
    
    # obtient les 10 correspondances les plus proches de notre chaîne d'entrée
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # ne récupère que les correspondances dont le ratio est > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # obtenir les lignes de toutes les correspondances proches dans notre cadre de données
    rows_with_matches = df[column].isin(close_matches)

    # remplacer toutes les lignes avec des correspondances proches par les correspondances d'entrée 
    df.loc[rows_with_matches, column] = string_to_match
    
    # nous indique que la fonction est terminée
    print("All done!")

Maintenant que nous avons une fonction, nous pouvons la mettre à l'épreuve !

In [20]:
# utiliser la fonction que nous venons d'écrire pour remplacer les correspondances proches de "d.i khan" par "d.i khan"
replace_matches_in_column(df=suicide_attacks, column='City', string_to_match="d.i khan")

All done!


Vérifions à nouveau les valeurs uniques de notre colonne Ville et assurons-nous que nous avons correctement mis de l'ordre dans d.i khan.

In [21]:
# obtenir toutes les valeurs uniques dans la colonne 'Ville'
cities = suicide_attacks['City'].unique()

# les trier par ordre alphabétique et regarder de plus près
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=object)

excellent ! Maintenant nous n'avons plus que "d.i khan" dans notre dataframe et nous n'avons plus besoin de changer quoi que ce soit à la main.